# IKEA-óskalistar

Við ætlum að safna gögnum um óskalista á http://www.ikea.is/wish_lists/

Kóðinn keyrir á Python 2.7. 

Ef þú vilt keyra kóðann á vélinni þinni þarftu að hafa iPython uppsett: http://ipython.org/install.html


## Byrjum ballið!

Flytjum inn þá pakka sem við þurfum.

requests: http://docs.python-requests.org/en/latest/

scraperwiki: https://pypi.python.org/pypi/scraperwiki/0.5

lxml: http://lxml.de/

json: https://docs.python.org/2/library/json.html

Athugaðu að þú þarft að setja upp pakkana hjá þér. Einfaldast er að hafa sérstakt Python-sýndarumhverfi (e. _virtualenv_) og setja pakkana upp þar.  

Hér eru ágætar leiðbeiningar um það: http://pythoncentral.io/how-to-install-virtualenv-python/


In [1]:
# -*- coding: utf-8 -*-
import requests
import scraperwiki
import lxml.html
import json

Skilgreinum nokkrar breytur

In [2]:
BASE_URL = 'http://www.ikea.is/wish_lists/%s'

LAST = 12000

s = requests.Session()

`BASE_URL` er slóðin með óskalistunum. Í lok slóðarinnar er einkvæmt númer hvers lista.

`LAST` er síðasta númer óskalista sem við ætlum að biðja um.  

Dæmi: https://www.ikea.is/wish_lists/10000

Við notum `requests.Session()` til að endurnýta HTTP-tenginguna á milli fyrirspurna. Það flýtir fyrir.

In [8]:
def scrape(id):
    response = s.get(BASE_URL % id)
    html = response.text
    root = lxml.html.fromstring(html)
    user_dict = {}
    user_products = []
    product_list_batch = []
    user_dict['id'] = id
    user_dict['name'] = root.xpath('//div[@class="sidebar_outer head first"]/h1')[0].text
    products = root.xpath('//div[@class="product_container"]')
    for product in products:
        product_dict = {}
        id = product.xpath('a')[1].attrib['href'].strip().replace('/products/', '')
        user_products.append(id)
        product_dict['id'] = id
        product_dict['title'] = product.xpath('a/div[@class="product_name"]')[0].text
        product_dict['price'] = product.xpath('div[@class="product_price"]')[0].text.replace('.', '').replace('kr', '').strip()
        product_list_batch.append(product_dict)
    user_dict['products'] = json.dumps(user_products)
    user = [user_dict]
    scraperwiki.sqlite.save(unique_keys=['id'], data=product_list_batch, table_name='products')
    scraperwiki.sqlite.save(unique_keys=['id'], data=user, table_name='users')
    

Skilgreinum fallið `scrape` sem mun sækja hvern óskalista. Það tekur óskalistanúmer sem færibreytu (e. _parameter_).

Skoðum þetta aðeins nánar:


    response = s.get(BASE_URL % id)
    html = response.text
    root = lxml.html.fromstring(html)

Sækjum HTML-kóða óskalistans og þáttum yfir í XML með `lxml`

    user_dict = {}
    user_products = []
    product_list_batch = []
    user_dict['id'] = id

Búum til hluti til að geyma gögnin sem við erum að fara að skrapa. 

    user_dict['name'] = root.xpath('//div[@class="sidebar_outer head first"]/h1')[0].text

Finnum nafn þess sem bjó óskalistann til. Til þess notum við XPath-segð (`//div[@class="sidebar_outer head first"]/h1`)

Hér er ágætt yfirlit yfir XPath: http://www.w3schools.com/xsl/xpath_intro.asp

Á mannamáli er þetta svona:

Byrjaðu efst í XML-skjalinu og finndu allt sem uppfyllir skilyrðin
Finndu `div`-einingu sem er með CSS-klasann `sidebar_outer head first`
Inni í henni skaltu finna tagið `h1`
Skilaðu okkur textanum í fyrsta slíka hlutnum sem þú finnur í listanum


    products = root.xpath('//div[@class="product_container"]')

Hér finnum við lista yfir allar vörurnar. Þær eru í `div`-einingum með CSS-klasann `product_container`

    for product in products:
        product_dict = {}
        id = product.xpath('a')[1].attrib['href'].strip().replace('/products/', '')
        user_products.append(id)
        product_dict['id'] = id
        product_dict['title'] = product.xpath('a/div[@class="product_name"]')[0].text
        product_dict['price'] = product.xpath('div[@class="product_price"]')[0].text.replace('.', '').replace('kr', '').strip()
        product_list_batch.append(product_dict)

Ítrum yfir vörulistann og geymum upplýsingar um hverja vöru

    user_dict['products'] = json.dumps(user_products)
    user = [user_dict]
    scraperwiki.sqlite.save(unique_keys=['id'], data=product_list_batch, table_name='products')
    scraperwiki.sqlite.save(unique_keys=['id'], data=user, table_name='users')

Vistum í tvær töflur í gagnagrunninum. 

`products` geymir upplýsingar um vörurnar.
`users` geymir upplýsingar um notendur.

Svona líta færslur í töflunni `users` út:

    `SELECT * FROM users LIMIT 2`

    "products", "id", "name"
    "[""12067""]",2,"Matti"
    "[""1481"", ""11981""]",3,"Steinunn Vigdís Steindórsdóttir"
    
Og svona í `products`:

    `SELECT * FROM products LIMIT 3`
    
    "price", "id", "title"
    785,38793,"STICKAT vasi á rúm 39x30 grænt"
    485,38809,"IKEA 365+ djúpur diskur/skál 17 cm hallandi hliðar hvítt"
    2750,39467," ÖNSKEDRÖM mynd 52x52 "


In [ ]:
try:
    last_done = scraperwiki.sqlite.select("id from users order by id DESC")
    last_done = last_done[0]['id']
except:
    last_done = 1

print "Last done: %s" % last_done

for i in range(last_done, LAST+1):
    try:
        scrape(i)
        print 'Done', i
    except:
        print 'FAIL for', i
        continue

Og svo keyrum við af stað.

Byrjum á að athuga hvort við höfum einhverja notendur í gagnagrunninum og sækjum þá `id` númer notandans (sem er hlaupandi tala). Ef ekki byrjum við á númer 1.

Svo sækjum við alla óskalistana frá seinasta notaða númeri og upp að `LAST` númerinu. Veitum smá upplýsingar um hvað er að gerast, en aðallega keyrum við bara áfram.